In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('아파트(전월세)_실거래가_전처리완료.csv')
df = df[['주소', '시도', '시군구', '읍면동', '번지', '도로명', '단지명']]
df

/var/folders/sz/jckrc5g14_5650t4yldl5ww40000gn/T/ipykernel_46492/2379410697.py:1: DtypeWarning: Columns (14,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('아파트(전월세)_실거래가_전처리완료.csv')


,주소,시도,시군구,읍면동,번지,도로명,단지명
0,서울특별시 서대문구 홍은동,서울특별시,서대문구,홍은동,464,통일로 510,북한산더샵
1,서울특별시 마포구 신공덕동,서울특별시,마포구,신공덕동,170,백범로 230,브라운스톤공덕아파트
2,서울특별시 동작구 상도동,서울특별시,동작구,상도동,363-164,상도로15길 131,상도휴엔하임
3,서울특별시 영등포구 신길동,서울특별시,영등포구,신길동,365,도신로 68,우성1
4,서울특별시 관악구 봉천동,서울특별시,관악구,봉천동,863-9,남부순환로220길 29,코스모블루
...,...,...,...,...,...,...,...
1442676,서울특별시 광진구 광장동,서울특별시,광진구,광장동,148-4,아차산로78길 10,워커힐푸르지오
1442677,서울특별시 동대문구 전농동,서울특별시,동대문구,전농동,6,사가정로 190,전농우성
1442678,서울특별시 성동구 행당동,서울특별시,성동구,행당동,347,행당로 79,대림e-편한세상(임대)
1442679,서울특별시 성동구 하왕십리동,서울특별시,성동구,하왕십리동,258,무학로 50,벽산


In [9]:
df_street = df[['시도', '시군구', '도로명']]
df_street['주소'] = df['시도'] + ' ' + df['시군구'] + ' ' + df['도로명']
df_street = df_street.groupby('주소').count().reset_index()
df_street = df_street.drop(['시도', '시군구', '도로명'], axis=1)
df_street

/var/folders/sz/jckrc5g14_5650t4yldl5ww40000gn/T/ipykernel_68816/301228240.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_street['주소'] = df['시도'] + ' ' + df['시군구'] + ' ' + df['도로명']


,주소
0,서울특별시 강남구
1,서울특별시 강남구 강남대로 240
2,서울특별시 강남구 강남대로 256
3,서울특별시 강남구 강남대로 272
4,서울특별시 강남구 강남대로 324
...,...
9531,서울특별시 중랑구 중랑천로12길 42
9532,서울특별시 중랑구 중랑천로14길 34
9533,서울특별시 중랑구 중랑천로14길 58
9534,서울특별시 중랑구 중랑천로2길 48


In [ ]:
from geopy.geocoders import Nominatim

# Initialize Nominatim geocoder
geolocator = Nominatim(user_agent="South Korea")

# Function to get latitude and longitude from an address
def get_lat_lon_nominatim(address):
    try:
        location = geolocator.geocode(address, timeout=15)
        if location:
            print('\n')
            print(f"주소: {address}")
            print(f"위도: {location.latitude}, 경도: {location.longitude}")
            return location.latitude, location.longitude
        else:
            print('\n')
            print("주소를 찾을 수 없습니다.")
            return None, None
    except:
        return None, None

# Example to get coordinates for a list of addresses
addresses = df_street['주소']

coordinates = [get_lat_lon_nominatim(address) for address in addresses]

coords_df = pd.DataFrame(coordinates, columns=['위도', '경도'])

In [11]:
df_street = pd.concat([df_street, coords_df], axis=1)
df_street

,주소,위도,경도
0,서울특별시 강남구,37.517700,127.047300
1,서울특별시 강남구 강남대로 240,37.485316,127.034433
2,서울특별시 강남구 강남대로 256,37.486542,127.033438
3,서울특별시 강남구 강남대로 272,37.487781,127.032938
4,서울특별시 강남구 강남대로 324,37.492168,127.030833
...,...,...,...
9531,서울특별시 중랑구 중랑천로12길 42,37.593317,127.075314
9532,서울특별시 중랑구 중랑천로14길 34,37.594472,127.074796
9533,서울특별시 중랑구 중랑천로14길 58,37.594545,127.075821
9534,서울특별시 중랑구 중랑천로2길 48,37.586736,127.077271


In [12]:
df_street.to_csv('아파트(전월세)_위경도.csv', index=False)

In [3]:
df_ground = df[['시도', '시군구', '읍면동', '번지']]
df_ground['주소'] = df['시도'] + ' ' + df['시군구'] + ' ' + df['읍면동'] + ' ' + df['번지']
df_ground = df_ground.groupby('주소').count().reset_index()
df_ground = df_ground.drop(['시도', '시군구', '읍면동', '번지'], axis=1)
df_ground

/var/folders/sz/jckrc5g14_5650t4yldl5ww40000gn/T/ipykernel_46492/1568733000.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ground['주소'] = df['시도'] + ' ' + df['시군구'] + ' ' + df['읍면동'] + ' ' + df['번지']


,주소
0,서울특별시 강남구 개포동 1164-12
1,서울특별시 강남구 개포동 1164-13
2,서울특별시 강남구 개포동 1164-14
3,서울특별시 강남구 개포동 1164-20
4,서울특별시 강남구 개포동 1164-25
...,...
9364,서울특별시 중랑구 중화동 438
9365,서울특별시 중랑구 중화동 450
9366,서울특별시 중랑구 중화동 452
9367,서울특별시 중랑구 중화동 453


In [4]:
from geopy.geocoders import Nominatim

# Initialize Nominatim geocoder
geolocator = Nominatim(user_agent="South Korea")

# Function to get latitude and longitude from an address
def get_lat_lon_nominatim(address):
    try:
        location = geolocator.geocode(address, timeout=15)
        if location:
            print('\n')
            print(f"주소: {address}")
            print(f"위도: {location.latitude}, 경도: {location.longitude}")
            return location.latitude, location.longitude
        else:
            print('\n')
            print("주소를 찾을 수 없습니다.")
            return None, None
    except:
        return None, None

# Example to get coordinates for a list of addresses
addresses = df_ground['주소']

coordinates = [get_lat_lon_nominatim(address) for address in addresses]

coords_df = pd.DataFrame(coordinates, columns=['위도', '경도'])



주소: 서울특별시 강남구 개포동 1164-12
위도: 37.48421, 경도: 127.07006


주소: 서울특별시 강남구 개포동 1164-13
위도: 37.48421, 경도: 127.07006


주소: 서울특별시 강남구 개포동 1164-14
위도: 37.48421, 경도: 127.07006


주소: 서울특별시 강남구 개포동 1164-20
위도: 37.48421, 경도: 127.07006


주소: 서울특별시 강남구 개포동 1164-25
위도: 37.48421, 경도: 127.07006


주소: 서울특별시 강남구 개포동 1164-28
위도: 37.48421, 경도: 127.07006


주소: 서울특별시 강남구 개포동 1164-30
위도: 37.48421, 경도: 127.07006


주소: 서울특별시 강남구 개포동 1164-7
위도: 37.48421, 경도: 127.07006


주소: 서울특별시 강남구 개포동 1165
위도: 37.48421, 경도: 127.07006


주소: 서울특별시 강남구 개포동 1167
위도: 37.48421, 경도: 127.07006


주소: 서울특별시 강남구 개포동 12
위도: 37.4827757, 경도: 127.061998


주소: 서울특별시 강남구 개포동 12-2
위도: 37.48421, 경도: 127.07006


주소: 서울특별시 강남구 개포동 1242-2
위도: 37.48421, 경도: 127.07006


주소: 서울특별시 강남구 개포동 1260-11
위도: 37.48421, 경도: 127.07006


주소: 서울특별시 강남구 개포동 1260-2
위도: 37.48421, 경도: 127.07006


주소: 서울특별시 강남구 개포동 1260-4
위도: 37.48421, 경도: 127.07006


주소: 서울특별시 강남구 개포동 1280
위도: 37.48421, 경도: 127.07006


주소: 서울특별시 강남구 개포동 1281
위도: 37.48421, 경도: 127.07006


주소: 서울특별시 강

In [5]:
df_ground = pd.concat([df_ground, coords_df], axis=1)
df_ground

,주소,위도,경도
0,서울특별시 강남구 개포동 1164-12,37.48421,127.07006
1,서울특별시 강남구 개포동 1164-13,37.48421,127.07006
2,서울특별시 강남구 개포동 1164-14,37.48421,127.07006
3,서울특별시 강남구 개포동 1164-20,37.48421,127.07006
4,서울특별시 강남구 개포동 1164-25,37.48421,127.07006
...,...,...,...
9364,서울특별시 중랑구 중화동 438,37.59755,127.07783
9365,서울특별시 중랑구 중화동 450,37.59755,127.07783
9366,서울특별시 중랑구 중화동 452,37.59755,127.07783
9367,서울특별시 중랑구 중화동 453,37.59755,127.07783


In [6]:
df_ground.to_csv('아파트(전월세)_위경도_지번.csv', index=False)